In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, LassoCV

from sklearn.ensemble import RandomForestRegressor

from sklearn.ensemble import AdaBoostRegressor

from sklearn.feature_extraction.text import CountVectorizer

from google.colab import drive, files

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
train = pd.read_csv('drive/MyDrive/capstone_project/train_cleaned.csv').sample(n=1500).reset_index(drop=True)

In [30]:
train.head()

,job_description,job_desig,key_skills,location,min_experience,max_experience,average_salary
0,should posse problem solving skill hand on dev...,LTI Invites SAS DI Professionals in Delhi!!- F...,"SAS, SQL, SAS DI Studio, SAS DI, SAS DQ, Base ...",Ghaziabad,4,9,6.0
1,experience of social campaign management and c...,"Marketing Executive ( SEO, SMM and Database Ma...","SEO, SEM, SMM, Online Marketing, PPC, Google A...",Noida(Sector-125 Noida),1,2,1.5
2,not_specified,Teradata BI,"Unix, Shell scripting, SDLC, Performance tunin...",Chennai,3,5,11.0
3,mba with 7 12 yr of experience mba with 7 12 y...,Brand Manager-private Label Staples Category B...,"Direct Marketing, Brand Management, Brand Posi...",Mumbai,7,12,27.5
4,not_specified,Marketing Specialist,"marketing, advertising, campaigns, activation,...",Bengaluru,3,5,6.0


In [31]:
train.isnull().sum()

job_description    0
job_desig          0
key_skills         0
location           0
min_experience     0
max_experience     0
average_salary     0
dtype: int64

In [32]:
train['cv_data'] = train["job_description"] + train['job_desig'] + train['key_skills'] + train['location']

In [33]:
cv = CountVectorizer()

In [34]:
cv.fit(train['cv_data'])

CountVectorizer()

In [35]:
cv_stringdata = cv.transform(train['cv_data'])

In [36]:
string_df = pd.DataFrame(cv_stringdata.todense(), columns=cv.get_feature_names_out())

In [37]:
string_df.head()

,000,02nd,08,10,100,10avp,10g,10marketing,10walk,10yrs,11,11pm,11project,11scientist,11sql,12,125,127,12yrs,13,132,14,142,142deep,14yrs,15,1500,15pm,15years,16,17,18,18001,1must,1principal,1project,1q,1st,2008,2016,...,xml,xsl,xslt,yamunanagar,yang,yarn,year,yearam,yearanalyst,yearbusiness,yearconsultant,yearfp,yearhris,yearibm,yearpre,years,yearsalesforce,yearsba,yearsdatabase,yearsenior,yearsr,yerwada,yerwadasas,yes,yield,you,youd,youhead,youll,youopening,your,youtube,yr,yrinternational,yrs,yrscost,yrsenior,zend,zoology,zotdevelopment
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [38]:
string_df.isnull().sum().sort_values(ascending=False)

zotdevelopment         0
executivehtml          0
executiveaccounting    0
executivead            0
executivebusiness      0
                      ..
oops                   0
ooty                   0
oozie                  0
open                   0
000                    0
Length: 5930, dtype: int64

In [39]:
string_df.shape

(1500, 5930)

In [40]:
train.shape

(1500, 8)

In [41]:
train.head().index

RangeIndex(start=0, stop=5, step=1)

In [42]:
quantified_df = pd.concat([string_df, train[['max_experience', 'min_experience', 'average_salary']]], axis=1, ignore_index=False)

In [43]:
quantified_df.isnull().sum()

000               0
02nd              0
08                0
10                0
100               0
                 ..
zoology           0
zotdevelopment    0
max_experience    0
min_experience    0
average_salary    0
Length: 5933, dtype: int64

In [44]:
quantified_df.shape

(1500, 5933)

In [45]:
quantified_df.head()

,000,02nd,08,10,100,10avp,10g,10marketing,10walk,10yrs,11,11pm,11project,11scientist,11sql,12,125,127,12yrs,13,132,14,142,142deep,14yrs,15,1500,15pm,15years,16,17,18,18001,1must,1principal,1project,1q,1st,2008,2016,...,yamunanagar,yang,yarn,year,yearam,yearanalyst,yearbusiness,yearconsultant,yearfp,yearhris,yearibm,yearpre,years,yearsalesforce,yearsba,yearsdatabase,yearsenior,yearsr,yerwada,yerwadasas,yes,yield,you,youd,youhead,youll,youopening,your,youtube,yr,yrinternational,yrs,yrscost,yrsenior,zend,zoology,zotdevelopment,max_experience,min_experience,average_salary
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,4,6.0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,1.5
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,3,11.0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,12,7,27.5
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,3,6.0


In [46]:
X = quantified_df.drop(columns='average_salary')
y = quantified_df['average_salary']

In [47]:
y.value_counts(normalize=True)

17.5    0.236667
27.5    0.232000
1.5     0.168667
11.0    0.168000
6.0     0.132667
50.0    0.062000
Name: average_salary, dtype: float64

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.33,
                                                    stratify=y,
                                                    random_state=42)

In [49]:
pipe = Pipeline([
    ('ss', StandardScaler()), 
    ('lin', LinearRegression())
])

In [50]:
pipe.fit(X_train, y_train)
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

(0.9734734872733232, -2.575613217303164e+26)

In [51]:
pipe2 = Pipeline([
    ('ss', StandardScaler()), 
    ('knn', KNeighborsRegressor())
])

In [52]:
pipe2.fit(X_train, y_train)
pipe2.score(X_train, y_train), pipe2.score(X_test, y_test)

(0.3759487394973844, -0.022805584612106244)

In [53]:
# pipe3 = Pipeline([
#     ('ss', StandardScaler()), 
#     ('lassoCV', LassoCV())
# ])

In [54]:
# pipe3.fit(X_train, y_train)
# pipe3.score(X_train, y_train), pipe3.score(X_test, y_test)

KeyboardInterrupt: ignored

In [55]:
pipe4 = Pipeline([
    ('ss', StandardScaler()), 
    ('rfr', RandomForestRegressor())
])

In [56]:
pipe4.fit(X_train, y_train)
pipe4.score(X_train, y_train), pipe4.score(X_test, y_test)

(0.921717295078884, 0.528130999810769)

In [58]:
pipe5 = Pipeline([
    ('ss',StandardScaler()), 
    ('abr', AdaBoostRegressor())
])

In [59]:
pipe5.fit(X_train, y_train)
pipe5.score(X_train, y_train), pipe5.score(X_test, y_test)

(0.4108934312679495, 0.37927918389629145)

In [60]:
results = {'Model': ['LinearRegression',
                     'KNeighborsRegressor',
                     #'LassoCV',
                     'RandomForestRegressor',
                     'AdaBoostRegressor'],
           'train_score': [pipe.score(X_train, y_train),
                           pipe2.score(X_train, y_train),
                           #pipe3.score(X_train, y_train), 
                           pipe4.score(X_train, y_train), 
                           pipe5.score(X_train, y_train)],
           'test_score': [pipe.score(X_test, y_test),
                           pipe2.score(X_test, y_test),
                           #pipe3.score(X_test, y_test), 
                           pipe4.score(X_test, y_test), 
                           pipe5.score(X_test, y_test)]
          }

In [61]:
results_df = pd.DataFrame(results)

In [62]:
results_df

,Model,train_score,test_score
0,LinearRegression,0.973473,-2.575613e+26
1,KNeighborsRegressor,0.375949,-2.280558e-02
2,RandomForestRegressor,0.921717,5.281310e-01
3,AdaBoostRegressor,0.410893,3.792792e-01
